In [ ]:
## Import Libraries ##
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math as m

from sklearn.metrics import mean_squared_error
from sklearn import metrics, model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision.models as models

import imageio
from tqdm.notebook import tqdm, trange
import PIL

/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Sending to CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Preprocessing

**Importing Cleaned Data**

**Data Augmentation**

In [ ]:
# (from deep_learning_2.ipynb, Dr. David E. Carlson)

# training data
data_transforms = transforms.Compose([ #data augmentation step 
        transforms.RandomResizedCrop(224), #random croppings of the image
        transforms.RandomHorizontalFlip(), #mirror image (usually not a random vertical flip)
        transforms.RandomRotation(10), #rotates image
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# test data
data_transforms_test = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #our new images must match statistics of original data
])

In [ ]:
# Data loader (from deep_learning_2.ipynb, Dr. David E. Carlson)

train_data = datasets.ImageFolder('/work/meb135/cleandata/train', transform=data_transforms)
val_data = datasets.ImageFolder('/work/meb135/cleandata/val', transform=data_transforms_test)
test_data = datasets.ImageFolder('/work/meb135/cleandata/test', transform=data_transforms_test)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, num_workers=0)

FileNotFoundError: [Errno 2] No such file or directory: '/work/meb135/cleandata/train'

## Model selection

In [ ]:
# Implementing AlexNet (WITH TRANSFER LEARNING)

modelTypes = ['AlexNet', 'VGG', 'ResNet', 'SqueezeNet', 'DenseNet', 'Inception v3', 'GoogleNet', 'ShuffleNet v2']
modelArr = ['model_ft = models.alexnet(pretrained=True)', 'model_ft = models.vgg16(pretrained=True)', /
'model_ft = models.resnet18(pretrained=True)', 'model_ft = models.squeezenet1_0(pretrained=True)', 'model_ft = models.densenet161(pretrained=True)' /
'model_ft = models.inception_v3(pretrained=True)', 'model_ft = models.googlenet(pretrained=True)', 'model_ft = models.shufflenet_v2_x1_0(pretrained=True)']
modelValAcc = []

In [ ]:
# Sending model to CUDA device
model_ft.to(device)

# Correcting number of classes to 2 (from deep_learning_2.ipynb)
num_ftrs = model_ft.classifier[-1].in_features # Need to know # of features coming out of the penultimate layer
model_ft.classifier[-1] = nn.Linear(num_ftrs, 1) # Redefining last feature to predict binary
model_ft.to(device)

In [ ]:
# Run CNN (from deep_learning_2.ipynb)

criterion = nn.BCEWithLogitsLoss() # binary cross entropy loss
optimizer = torch.optim.SGD(model_ft.parameters(), lr=1e-5, momentum=0.9) # momentum smooths the dataset (impacts stochastic gradient descent)

def test_model():
    ## Testing
    correct = 0
    total = len(test_data)
    model_ft.eval() # set the model into evaluation mode, which changes the 
    # behavior of the batch norm layer so that it is not sensitive to batch size
    with torch.no_grad():
        # Iterate through test set minibatchs 
        for images, labels in tqdm(test_loader):
            # Forward pass
            inputs = images.to(device)
            labels = labels.unsqueeze(1).to(device)
            y = model_ft(inputs)

            predictions = torch.round(torch.sigmoid(y)).long()#torch.argmax(y, dim=1)
            correct += torch.sum((predictions == labels).float())
    print('Test accuracy: {}'.format(correct/total))


def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in trange(num_epochs):
        for images, labels in tqdm(train_loader):

            inputs = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Do the forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Calculate gradients and step (update parameters)
            loss.backward()
            optimizer.step()
        test_model()
        model_ft.train() # set model back to training mode (batch norm layers back to normal) after testing



In [ ]:
modelTypes = ['AlexNet', 'VGG', 'ResNet', 'SqueezeNet', 'DenseNet', 'Inception v3', 'GoogleNet', 'ShuffleNet v2']
modelArr = ['model_ft = models.alexnet(pretrained=True)', 'model_ft = models.vgg16(pretrained=True)', /
'model_ft = models.resnet18(pretrained=True)', 'model_ft = models.squeezenet1_0(pretrained=True)', 'model_ft = models.densenet161(pretrained=True)' /
'model_ft = models.inception_v3(pretrained=True)', 'model_ft = models.googlenet(pretrained=True)', 'model_ft = models.shufflenet_v2_x1_0(pretrained=True)']
modelValAcc = []

def test_model():
    ## Testing
    correct = 0
    total = len(test_data)
    model_ft.eval() # set the model into evaluation mode, which changes the 
    # behavior of the batch norm layer so that it is not sensitive to batch size
    with torch.no_grad():
        # Iterate through test set minibatchs 
        for images, labels in tqdm(test_loader):
            # Forward pass
            inputs = images.to(device)
            labels = labels.unsqueeze(1).to(device)
            y = model_ft(inputs)

            predictions = torch.round(torch.sigmoid(y)).long()#torch.argmax(y, dim=1)
            correct += torch.sum((predictions == labels).float())
    print('Test accuracy: {}'.format(correct/total))


def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in trange(num_epochs):
        for images, labels in tqdm(train_loader):

            inputs = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Do the forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Calculate gradients and step (update parameters)
            loss.backward()
            optimizer.step()
        test_model()
        model_ft.train() # set model back to training mode (batch norm layers back to normal) after testing

#iterate here!!
for m in modelArr:
    model_ft = eval(m)
    
    num_ftrs = model_ft.classifier[-1].in_features # Need to know # of features coming out of the penultimate layer
    model_ft.classifier[-1] = nn.Linear(num_ftrs, 1) # Redefining last feature to predict binary
    model_ft.to(device)

    criterion = nn.BCEWithLogitsLoss() # binary cross entropy loss
    optimizer = torch.optim.SGD(model_ft.parameters(), lr=1e-5, momentum=0.9) # momentum smooths the dataset (impacts stochastic gradient descent)
    
    train_model(model_ft, criterion, optimizer, num_epochs=3)

SyntaxError: invalid syntax (<ipython-input-1-46902d1f0964>, line 2)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeea15ee-1713-4850-a1d6-fce82f9c7017' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>